In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

/Users/ruthparajo/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


### Unweighted day-normalized

In [5]:
# Load energy data
energy = pd.read_csv('data/fasce_monthly_IT012E00314418_full.csv')
energy['date'] = pd.to_datetime(energy['date'])

# Load CDD data
cdd = pd.read_csv('data/LIML_CDD_24C.csv', skiprows=6)
cdd['Month starting'] = pd.to_datetime(cdd['Month starting'])
cdd = cdd[['Month starting', 'CDD 24']]

# Load HDD data
hdd = pd.read_csv('data/LIML_HDD_23C.csv', skiprows=6)
hdd['Month starting'] = pd.to_datetime(hdd['Month starting'])
hdd = hdd[['Month starting', 'HDD 23']]

# Merge all data on month/date
df = pd.merge(energy, cdd, left_on='date', right_on='Month starting', how='inner')
df = pd.merge(df, hdd, left_on='date', right_on='Month starting', how='inner')
# Filter df between 2023 and 2024
df = df[(df['date'] >= '2023-01-01') & (df['date'] <= '2024-12-31')]

# Prepare regression variables
X = df[['days_in_month', 'HDD 23', 'CDD 24']]
y = df['total_kWh']
#X = sm.add_constant(X)  # Adds intercept

# Fit regression model
model = sm.OLS(y, X).fit()
print(model.summary())

                                 OLS Regression Results                                
Dep. Variable:              total_kWh   R-squared (uncentered):                   0.984
Model:                            OLS   Adj. R-squared (uncentered):              0.982
Method:                 Least Squares   F-statistic:                              435.9
Date:                Wed, 17 Sep 2025   Prob (F-statistic):                    4.60e-19
Time:                        09:54:08   Log-Likelihood:                         -227.05
No. Observations:                  24   AIC:                                      460.1
Df Residuals:                      21   BIC:                                      463.6
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                    coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------

In [6]:
# Use the model-aligned data and counts
resid = model.resid                     # residuals on the training rows actually used
y_bar = model.model.endog.mean()        # mean of y on those same rows
n = int(model.nobs)                     # number of obs actually used
p = len(model.params)                   # number of estimated parameters (incl. intercept if present)

nmbe = resid.sum() / ((n - p) * y_bar) * 100
rmse = np.sqrt((resid**2).sum() / (n - p))
cvrmse = rmse / y_bar * 100



print("Model Evaluation Metrics")
print(f"R²:      {model.rsquared:.3f}")
print(f"NMBE:    {nmbe:.2f}% (target: |≤5% monthly|)")
print(f"CV(RMSE):{cvrmse:.2f}% (target: ≤15% monthly)")

Model Evaluation Metrics
R²:      0.984
NMBE:    0.11% (target: |≤5% monthly|)
CV(RMSE):13.74% (target: ≤15% monthly)


### Validation

In [7]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, r2_score

def compute_metrics(y_true, y_pred, p):
    resid = y_true - y_pred
    n = len(y_true)

    # R²
    r2 = r2_score(y_true, y_pred)

    # RMSE
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))

    # CV(RMSE) (%)
    cvrmse = rmse / y_true.mean() * 100

    # NMBE (%)
    nmbe = resid.sum() / ((n - p) * y_true.mean()) * 100

    return {"R²": r2, "CV(RMSE)": cvrmse, "NMBE": nmbe}

results = {}

for year in df['date'].dt.year.unique():
    # Train on all years except this one
    train = df[df['date'].dt.year != year]
    test = df[df['date'].dt.year == year]

    X_train = train[['days_in_month', 'HDD 23', 'CDD 24']]
    y_train = train['total_kWh']

    X_test = test[['days_in_month', 'HDD 23', 'CDD 24']]
    y_test = test['total_kWh']

    # Fit model (no intercept, to match your setup)
    model = sm.OLS(y_train, X_train).fit()

    # Predictions
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)

    # Metrics (p = number of predictors = 3 here)
    train_metrics = compute_metrics(y_train, y_pred_train, p=X_train.shape[1])
    test_metrics = compute_metrics(y_test, y_pred_test, p=X_test.shape[1])

    results[year] = {"train": train_metrics, "test": test_metrics}

# Show results
for year, metrics in results.items():
    print(f"\n=== Leaving out {year} ===")
    print("Train:", metrics["train"])
    print(" Test:", metrics["test"])



=== Leaving out 2023 ===
Train: {'R²': 0.832179817771866, 'CV(RMSE)': 9.188871658408614, 'NMBE': 0.0957195690320159}
 Test: {'R²': 0.20982179562970737, 'CV(RMSE)': 16.82563613283516, 'NMBE': -7.603126114512658}

=== Leaving out 2024 ===
Train: {'R²': 0.3642820767403744, 'CV(RMSE)': 15.091805434406966, 'NMBE': 0.18380705872566458}
 Test: {'R²': 0.7064539522392257, 'CV(RMSE)': 12.152852037187047, 'NMBE': 7.954801247546088}
